In [2]:
# Experiment Parameters

# Experiment conditions
# rewriter rules
use_all_rewriter_rules = True

# bigraph method
convert_bigraph = True

# NOT box model
not_representation = "X"
not_placement = "end"

# train NOT sentences
train_not_sentences = True

# load params
train_params = False
load_params = False

# training hyperparams
# SPSA hyperparams
spsa_a = 0.2
spsa_c = 0.06
# gamma = 0.928382463478119
spsa_n_iter = 130

# SVM type
kernel = 'rbf'
default_svm = True
optimize_svm = False

# Experiment metadata
experiment_name = "coherence_measurement"
verbose = True

In [3]:
import os

if (experiment_name == ""):
    experiment_name = "misc"

path = '../data/experiment_results/'+experiment_name

try:
    os.mkdir(path)
except:
    print("experiment folder already exists")

In [4]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

In [5]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
import numpy as np

def read_data(filename):
    labels, sentences = [], []
    with open(filename) as f:
        for line in f:
            labels.append([1, 0] if line[0] == '1' else [0, 1])
            sentences.append(line[1:].strip())
    return np.array(labels), sentences

test_targets_src, test_data_src = read_data('../data/datasets/restaurant_v3_test.txt')

dev_targets_src, dev_data_src = read_data('../data/datasets/restaurant_v3_dev.txt')
train_targets_src, train_data_src = read_data('../data/datasets/restaurant_v3_train.txt')

In [7]:
# Helper function for converting to bigraph

from discopy.rigid import Id
''
def checkTrailingCups(diagram):
    scanWords = True
    
    for box in diagram.boxes:
        if not box.dom and not scanWords:
            return False
        else:
            scanWords = scanWords and not box.dom
    
    return True

def convertToTrailingCups(diagram):
    if (checkTrailingCups(diagram)):
        return diagram

    words = []
    cups = []
    
    for box in diagram.boxes:
        if not box.dom:
            words = words + [box]
        else:
            cups = [box] + cups
    
    new_diag = words[0]
    
    for i, word in enumerate(words):
        if i != 0:
            new_diag = new_diag >> Id(new_diag.cod) @ word
    
    for i, cup in enumerate(cups):
        if i != len(cups)-1:
            new_diag = new_diag >> Id(new_diag.cod[:-2]) @ cup
        else:
            new_diag = new_diag >> cup @ Id(new_diag.cod[2:])
    
    return new_diag

In [8]:
# Fucntion for stemming and lemmatization of tokens

def to_word_tokens(data):
    return [word_tokenize(record) for record in data]

def build_stem_dictionary(data):
    port = PorterStemmer()
    wnet = WordNetLemmatizer()
    
    mapping = {}
    
    data_as_tokens = to_word_tokens(data)
    
    for words in data_as_tokens:
        for word in words:
            if word not in mapping:
                stemmed_word = port.stem(word)
                lemmatized_word = wnet.lemmatize(stemmed_word)
                
                mapping[word] = lemmatized_word
    
    return mapping

In [9]:
# Function for stemming and lemmatization of diagram boxes

from lambeq.rewrite import RewriteRule

class StemRewriteRule(RewriteRule):
    def __init__(self, data):
        self.mapping = build_stem_dictionary(data)
    
    def matches(self, box):
        return box.name in self.mapping

    def rewrite(self, box):
        new_name = self.mapping[box.name]
        return type(box)(name=new_name, dom=box.dom, cod=box.cod)

In [10]:
from lambeq.ansatz import CircuitAnsatz
from abc import abstractmethod
# from collections.abc import Mapping
from itertools import cycle
from typing import Callable, Optional, Tuple, Mapping
from discopy.quantum.circuit import (Circuit, Discard, Functor, Id, qubit)
from discopy.quantum.gates import Bra, H, Ket, Rx, Ry, Rz
from discopy.rigid import Box, Diagram, Ty
from discopy.tensor import Dim, Tensor
import numpy as np
from sympy import Symbol, symbols

class SimpleSA(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, Rx, Rz, CX, Ry

        if n_qubits == 1:
            if len(params) == 0:
                circuit = Id(1)
            else:
                circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
        elif len(Tensor.np.shape(params)) != 2\
                or Tensor.np.shape(params)[1] != n_qubits - 1:
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits - 1))
        else:
            depth = Tensor.np.shape(params)[0]
            circuit = Id(n_qubits)

            for thetas in params:
                hadamards = Id().tensor(*(n_qubits * [H]))
                rotations = Id(n_qubits).then(*(
                    (Id(i) @ CX @ Id(n_qubits - 2 - i)) >> (Id(i + 1) @ Rz(thetas[i]) @ Id(n_qubits - 2 - i) >> (Id(i + 1) @ H @ Id(n_qubits - 2 - i)))
                    for i in range(n_qubits - 1)))
                circuit >>= hadamards >> rotations

        super().__init__(circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class SimpleSAAnsatz(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, discard: bool = False) -> None:
        super().__init__(ob_map, n_layers, n_single_qubit_params, SimpleSA, discard, [Rx, Rz], H)

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        return (self.n_layers, n_qubits - 1)

class Sim15SA(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import CX, Ry

        params_shape = Tensor.np.shape(params)

        if n_qubits == 1:
            circuit = Id()
        elif (len(params_shape) != 2) or (params_shape[1] != 2 * n_qubits):
            raise ValueError(
                "Expected params of shape (depth, {})".format(2 * n_qubits))
        else:
            depth = params_shape[0]
            circuit = Id(n_qubits)

            for thetas in params:
                sublayer1 = Id().tensor(*map(Ry, thetas[:n_qubits]))

                for i in range(n_qubits):
                    src = i
                    tgt = (i - 1) % n_qubits
                    sublayer1 = sublayer1.CX(src, tgt)

                sublayer2 = Id().tensor(*map(Ry, thetas[n_qubits:]))

                for i in range(n_qubits, 0, -1):
                    src = i % n_qubits
                    tgt = (i + 1) % n_qubits
                    sublayer2 = sublayer2.CX(src, tgt)

                circuit >>= sublayer1 >> sublayer2

        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class Sim15SAAnsatz(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, discard: bool = False) -> None:
        super().__init__(ob_map, n_layers, n_single_qubit_params, Sim15SA, discard, [Rx, Rz])

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        return (self.n_layers, 2 * n_qubits)

In [11]:
class HCRxLayer(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, CRx

        params_shape = Tensor.np.shape(params)

        if n_qubits == 1:
            # circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
            circuit = Id(1)
        elif (len(params_shape) != 2) or (params_shape[1] != n_qubits):
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits))
        else:
            depth = params_shape[0]
            circuit = Id(n_qubits)

            for thetas in params:
                sublayer1 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits):
                    src = i
                    tgt = (i - 1) % n_qubits
                    sublayer1 = sublayer1.CRx(thetas[i], src, tgt)

                sublayer2 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits, 0, -1):
                    src = i % n_qubits
                    tgt = (i + 1) % n_qubits
                    sublayer2 = sublayer2.CRx(thetas[-i], src, tgt)

                circuit >>= sublayer1 >> sublayer2

        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class HCRzLayer(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, CRz

        params_shape = Tensor.np.shape(params)

        if n_qubits == 1:
            # circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
            circuit = Id(1)
        elif (len(params_shape) != 2) or (params_shape[1] != n_qubits):
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits))
        else:
            depth = params_shape[0]
            circuit = Id(n_qubits)

            for thetas in params:
                sublayer1 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits):
                    src = i
                    tgt = (i - 1) % n_qubits
                    sublayer1 = sublayer1.CRz(thetas[i], src, tgt)

                sublayer2 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits, 0, -1):
                    src = i % n_qubits
                    tgt = (i + 1) % n_qubits
                    sublayer2 = sublayer2.CRz(thetas[-i], src, tgt)

                circuit >>= sublayer1 >> sublayer2

        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class HCRxLayerV2(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, CRx

        params_shape = Tensor.np.shape(params)

        if n_qubits == 1:
            # circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
            circuit = Id(1)
        elif (len(params_shape) != 2) or (params_shape[1] != 2*n_qubits):
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits))
        else:
            depth = params_shape[0]
            circuit = Id(n_qubits)

            for thetas in params:
                sublayer1 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits):
                    src = i
                    tgt = (i - 1) % n_qubits
                    sublayer1 = sublayer1.CRx(thetas[i], src, tgt)

                sublayer2 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits, n_qubits*2):
                    src = i % n_qubits
                    tgt = (i + 1) % n_qubits
                    sublayer2 = sublayer2.CRx(thetas[i], src, tgt)

                circuit >>= sublayer1 >> sublayer2

        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class HCRzLayerV2(Circuit):
    def __init__(self, n_qubits, params):
        from discopy.quantum.gates import H, CRz

        params_shape = Tensor.np.shape(params)

        if n_qubits == 1:
            # circuit = Rx(params[0]) >> Rz(params[1]) >> Rx(params[2])
            circuit = Id(1)
        elif (len(params_shape) != 2) or (params_shape[1] != 2*n_qubits):
            raise ValueError(
                "Expected params of shape (depth, {})".format(n_qubits))
        else:
            depth = params_shape[0]
            circuit = Id(n_qubits)

            for thetas in params:
                sublayer1 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits):
                    src = i
                    tgt = (i - 1) % n_qubits
                    sublayer1 = sublayer1.CRz(thetas[i], src, tgt)

                sublayer2 = Id().tensor(*([H for _ in range(n_qubits)]))

                for i in range(n_qubits, 2*n_qubits, -1):
                    src = i % n_qubits
                    tgt = (i + 1) % n_qubits
                    sublayer2 = sublayer2.CRz(thetas[i], src, tgt)

                circuit >>= sublayer1 >> sublayer2

        super().__init__(
            circuit.dom, circuit.cod, circuit.boxes, circuit.offsets)

class GeneralQCAnsatz(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, block_layer = HCRxLayer, discard: bool = False) -> None:
        super().__init__(ob_map, n_layers, n_single_qubit_params, block_layer, discard, [Rx, Rz], H)

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        return (self.n_layers, n_qubits)

class GeneralQCAnsatzV2(CircuitAnsatz):
    def __init__(self, ob_map: Mapping[Ty, int], n_layers: int, n_single_qubit_params: int = 0, block_layer = HCRxLayerV2, discard: bool = False) -> None:
        super().__init__(ob_map, n_layers, n_single_qubit_params, block_layer, discard, [Rx, Rz], H)

    def params_shape(self, n_qubits: int) -> Tuple[int, ...]:
        return (self.n_layers, 2*n_qubits)

In [12]:
from lambeq import BobcatParser, Rewriter, remove_cups
from discopy import grammar
from discopy.quantum.gates import X, Z

def sentences_to_circuits(sentences, ansatz, convert_bigraph=True, not_representation="X", all_sentences=None, return_valids_mask=True):
    ### SENTENCES TO DIAGRAMS ###

    if all_sentences is None:
        all_sentences = sentences
    
    # syntax tree parsing
    parser = BobcatParser()
    raw_diagrams = parser.sentences2diagrams([text.replace(" not ", " ") for text in sentences])

    # filter valid diagrams type S
    n_sent = len(sentences)

    valids_mask = np.array([d.cod == Ty('s') for d in raw_diagrams])
    data = [sentences[i] for i in range(n_sent) if valids_mask[i]]
    use_diagrams = [raw_diagrams[i] for i in range(n_sent) if valids_mask[i]]

    # bigraph method
    rewriter = Rewriter()
    rewritten_diagrams = [rewriter(diagram) for diagram in use_diagrams]

    normalised_diagrams = [convertToTrailingCups(diagram.normal_form()) for diagram in rewritten_diagrams]

    removed_diagrams = [remove_cups(diagram) for diagram in normalised_diagrams]

    # stemming and lemmatization
    stemmed_diagrams = [Rewriter([StemRewriteRule(all_sentences)])(diagram) for diagram in removed_diagrams]

    # final diagrams
    diagrams = [diagram for diagram in stemmed_diagrams]

    ### DIAGRAMS to CIRCUITS ###

    # string diagrams to raw quantum circuits
    circuits = [ansatz(diagram) for diagram in diagrams]

    # apply NOT box to circuits
    for i, circuit in enumerate(circuits):
        if data[i].find(" not ") != -1:
            if (not_representation == "ZX"):
                circuits[i] = circuit >> Z >> X
            else:
                circuits[i] = circuit >> X
    
    if return_valids_mask:
        return data, diagrams, circuits, valids_mask
    else:
        return data, diagrams, circuits

In [14]:
import os
import json

working_dir = "../data/experiment_results/"
experiment_book = []

for experiment_name in os.listdir(working_dir):
    experiment_dir = working_dir+experiment_name+"/"
    if os.path.isdir(experiment_dir):
        if "fit_time.txt" in os.listdir(experiment_dir):
            experiment_dict = {"name": experiment_name}

            with open(experiment_dir+"params.txt", "r") as f:
                content = f.read().replace("\'", "\"")
                params_dict = json.loads(content)
                f.close()
            
            experiment_dict["params"] = params_dict

            if "200iter" in experiment_name:
                experiment_dict["iter"] = 200
            else:
                experiment_dict["iter"] = 130
            
            if "2layer" in experiment_name:
                experiment_dict["n_layers"] = 2
            elif "3layer" in experiment_name:
                experiment_dict["n_layers"] = 3
            else:
                experiment_dict["n_layers"] = 1
            
            if "generalQC" in experiment_name:
                is_v2 = False
                if "V2" in experiment_name:
                    experiment_dict["ansatz"] = "GeneralQCAnsatzV2"
                    
                    if "H_CRx" in experiment_name:
                        experiment_dict["block_layer"] = "HCRxLayerV2"
                    elif "H_CRz" in experiment_name:
                        experiment_dict["block_layer"] = "HCRzLayerV2"
                else:
                    experiment_dict["ansatz"] = "GeneralQCAnsatz"

                    if "H_CRx" in experiment_name:
                        experiment_dict["block_layer"] = "HCRxLayer"
                    elif "H_CRz" in experiment_name:
                        experiment_dict["block_layer"] = "HCRzLayer"
            
            if "iqp" in experiment_name:
                experiment_dict["ansatz"] = "IQPAnsatz"
              
            if "simple_sa" in experiment_name:
                experiment_dict["ansatz"] = "SimpleSAAnsatz"

                if "with_nouns" in experiment_name:
                    experiment_dict["n_single_qubit_params"] = 3
                else:
                    experiment_dict["n_single_qubit_params"] = 0
            
            if "sim15" in experiment_name:
                experiment_dict["ansatz"] = "Sim15Ansatz"
            if "sim15_sa" in experiment_name:
                experiment_dict["ansatz"] = "Sim15SAAnsatz"
        
            experiment_book.append(experiment_dict)

In [16]:
from pytket.extensions.qiskit import AerBackend

backend = AerBackend()

backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'n_shots': 8192  # maximum recommended shots, reduces sampling error
}

In [17]:
from discopy.quantum import Circuit, Id, Measure

def randint(rng, low=-1 << 63, high=1 << 63-1):
    return rng.integers(low, high)

def normalise(predictions):
    # apply smoothing to predictions
    predictions = np.abs(predictions) + 1e-9
    return predictions / predictions.sum()

def make_pred_fn(circuits, parameters, rng):
    measured_circuits = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuits]
    circuit_fns = [c.lambdify(*parameters) for c in measured_circuits]

    def predict(params):
        outputs = Circuit.eval(*(c_fn(*params) for c_fn in circuit_fns),
                               **backend_config, seed=randint(rng))
        return np.array([normalise(output.array) for output in outputs])
    return predict

In [25]:
from lambeq import AtomicType, IQPAnsatz, Sim15Ansatz
from sympy import default_sort_key


def decapitate_circuit(circuit):
    from discopy import Bra, Measure

    cutoff = None

    for i, box in enumerate(circuit.boxes):
        if box in [Bra, Measure, Bra(0), Bra(0, 0)]:
            cutoff = i
            break
    
    return circuit[:i]

def build_circuit_library(experiment_dict, data_src, targets_src):
    name = experiment_dict["name"]
    params = experiment_dict["params"]
    ansatz_name = experiment_dict["ansatz"]
    n_layers = experiment_dict["n_layers"]
    spsa_n_iter = experiment_dict["iter"]

    if ansatz_name in ["SimpleSAAnsatz"]:
        n_single_qubit_params = experiment_dict["n_single_qubit_params"]
    if ansatz_name in ["GeneralQCAnsatz", "GeneralQCAnsatzV2"]:
        block_layer_name = experiment_dict["block_layer"]
    
    parameters = list(params.keys())
    param_vals = [params[key] for key in parameters]

    # build ansatz
    N = AtomicType.NOUN
    S = AtomicType.SENTENCE

    if ansatz_name == "IQPAnsatz":
        ansatz = IQPAnsatz({N: 1, S: 1}, n_layers=n_layers)
    elif ansatz_name == "SimpleSAAnsatz":
        ansatz = SimpleSAAnsatz({N: 1, S: 1}, n_layers=n_layers, n_single_qubit_params=n_single_qubit_params)
    elif ansatz_name == "Sim15Ansatz":
        ansatz = Sim15Ansatz({N: 1, S: 1}, n_layers=n_layers)
    elif ansatz_name == "Sim15SAAnsatz":
        ansatz = Sim15SAAnsatz({N: 1, S: 1}, n_layers=n_layers)
    elif ansatz_name == "GeneralQCAnsatz":
        if block_layer_name == "HCRxLayer":
            block_layer = HCRxLayer
        elif block_layer_name == "HCRzLayer":
            block_layer = HCRzLayer
        
        ansatz = GeneralQCAnsatz({N: 1, S: 1}, n_layers=n_layers, block_layer=block_layer)
    elif ansatz_name == "GeneralQCAnsatzV2":
        if block_layer_name == "HCRxLayerV2":
            block_layer = HCRxLayerV2
        elif block_layer_name == "HCRzLayerV2":
            block_layer = HCRzLayerV2
        
        ansatz = GeneralQCAnsatzV2({N: 1, S: 1}, n_layers=n_layers, block_layer=block_layer)
    
    data, diagrams, circuits, valids_mask = sentences_to_circuits(data_src, ansatz)
    targets = [target for i, target in enumerate(targets_src) if valids_mask[i]]

    circuit_heads = []

    # instantiate complete circuits

    SEED = 0
    rng = np.random.default_rng(SEED)

    use_parameters = sorted({s for circ in circuits for s in circ.free_symbols},key=default_sort_key)
    use_param_strs = [str(param_sym) for param_sym in use_parameters]
    use_param_vals = [val for i, val in enumerate(param_vals) if parameters[i] in use_param_strs]

    measured_circuits = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuits]
    circuit_fns = [c.lambdify(*use_parameters) for c in measured_circuits]
    instantiated_circuits = [c_fn(*use_param_vals) for c_fn in circuit_fns]

    outputs = Circuit.eval(*instantiated_circuits, **backend_config, seed=randint(rng))
    res = np.array([normalise(output.array) for output in outputs])

    circuit_heads = [decapitate_circuit(c) for c in circuits]

    # measured_circuit_heads = [c >> Id().tensor(*[Measure()] * len(c.cod)) for c in circuit_heads]
    circuit_head_fns = [c.lambdify(*use_parameters) for c in circuit_heads]
    instantiated_circuit_heads = [c_fn(*use_param_vals) for c_fn in circuit_head_fns]

    return data, targets, parameters, param_vals, diagrams, ansatz, circuits, instantiated_circuits, circuit_heads, instantiated_circuit_heads, outputs, res

In [19]:
def computational_bases(n_qubits):
    return np.array([[int(j==i) for j in range(2**n_qubits)] for i in range(2**n_qubits)], dtype='cfloat')

def tensor_prod(v2, v1):
    return np.tensordot(v1, v2, axes=0).flatten()

def bloch_bases(n_qubits, n_alpha, n_beta):
    z_1 = computational_bases(1)
    b_plus = lambda alpha, beta: np.cos(alpha/2)*z_1[0] + np.sin(alpha/2)*np.exp((0+1j)*beta)*z_1[1]
    b_minus = lambda alpha, beta: np.sin(alpha/2)*z_1[0] - np.cos(alpha/2)*np.exp((0+1j)*beta)*z_1[1]

    alpha_dom = np.linspace(0, np.pi, n_alpha)
    beta_dom = np.linspace(0, 2*np.pi, n_beta)

    def b_bases_paramx(thetas):
        b_bases = computational_bases(0)

        for i in range(n_qubits):
            b_pluses = [tensor_prod(b_plus(thetas[2*i], thetas[2*i+1]), b_basis) for b_basis in b_bases]
            b_minuses = [tensor_prod(b_minus(thetas[2*i], thetas[2*i+1]), b_basis) for b_basis in b_bases]

            b_bases = np.vstack([b_pluses, b_minuses])
        
        return b_bases
    
    bloch_space = [None for _ in range((n_alpha*n_beta)**(n_qubits))]

    def recursive_search(level, curr_idx=[], curr_thetas=[]):
        for i, alpha0 in enumerate(alpha_dom):
            for j, beta0 in enumerate(beta_dom):
                if level == 1:
                    index = 0
                    for a_i, b_j in curr_idx+[[i, j]]:
                        index = index*n_alpha*n_beta+a_i*n_alpha+b_j
                    
                    bloch_space[index] = b_bases_paramx(curr_thetas+[alpha0, beta0])
                else:
                    recursive_search(level-1, curr_idx+[[i, j]], curr_thetas+[alpha0, beta0])

    recursive_search(n_qubits)

    return bloch_space

def trace(mat):
    return np.cfloat(sum([mat[i][i] for i in range(len(mat))]))

def abstrace(mat):
    return float(sum([np.absolute(mat[i][i]) for i in range(len(mat))]))

def KD(a, b, rho):
    Pi_a = np.outer(a, np.conjugate(a))
    Pi_b = np.outer(b, np.conjugate(b))
    return trace(np.matmul(Pi_b, np.matmul(Pi_a, rho)))

In [20]:
from qiskit import QuantumCircuit, execute
from qiskit import Aer

def build_clifford_set(n_qubits, cliffords=[]):
    single_qubit_cliffords = [
        '',
        'H', 'S',
        'HS', 'SH', 'SS',
        'HSH', 'HSS', 'SHS', 'SSH', 'SSS',
        'HSHS', 'HSSH', 'HSSS', 'SHSS', 'SSHS',
        'HSHSS', 'HSSHS', 'SHSSH', 'SHSSS', 'SSHSS',
        'HSHSSH', 'HSHSSS', 'HSSHSS'
    ]

    if n_qubits == 0:
        return cliffords
    else:
        if len(cliffords) == 0:
            return build_clifford_set(n_qubits-1, [[sc] for sc in single_qubit_cliffords])
        else:
            temp_cliffords = [[c+[sc] for sc in single_qubit_cliffords] for c in cliffords]
            new_cliffords = [item for sublist in temp_cliffords for item in sublist]

            return build_clifford_set(n_qubits-1, new_cliffords)

def build_clifford_states(n_qubits):
    backend = Aer.get_backend('unitary_simulator')

    multi_qubit_cliffords = build_clifford_set(n_qubits)
    mqc_svs = [clifford_str_to_sv(c, backend) for c in multi_qubit_cliffords]

    return mqc_svs

def clifford_str_to_sv(clifford_str, backend):
    n_qubits = len(clifford_str)

    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        for j, gate in enumerate(clifford_str[i]):
            if gate == "H":
                qc.h(n_qubits-i-1)
            if gate == "S":
                qc.s(n_qubits-i-1)

    job = execute(qc, backend, shots=8192)
    job_result = job.result()

    return job_result.get_unitary(qc).data

In [21]:
multi_qubit_cliffords = {}

for n_qubits in [2, 3, 4]:
    multi_qubit_cliffords[n_qubits] = build_clifford_states(n_qubits)

In [22]:
import pickle

with open("../data/datasets/mqc_vals.pkl", "wb") as f:
    pickle.dump(multi_qubit_cliffords, f)

In [23]:
z_bases = {2: computational_bases(2), 3: computational_bases(3), 4: computational_bases(4)}

In [ ]:
import math
import time
from tqdm import tqdm

coherence_dict = {}

for k, experiment_dict in enumerate(experiment_book):
    if experiment_dict["name"] in coherence_dict:
        experiment_book[k]["NCLs"] = np.array(coherence_dict[experiment_dict["name"]]["ncls"])
        experiment_book[k]["coherence"] = coherence_dict[experiment_dict["name"]]["avg"]
        continue
    
    data, targets, parameters, param_vals, diagrams, ansatz, circuits, instantiated_circuits, circuit_heads, instantiated_circuit_heads, outputs, res = build_circuit_library(experiment_dict, test_data_src, test_targets_src)
    sv_data = [c.to_tk().get_statevector() for c in instantiated_circuit_heads]

    NCLs = [None for _ in sv_data]

    sv_lengths = [len(sv) for sv in sv_data]
    sv_sorted = [x for _, x in sorted(zip(sv_lengths, sv_data), key=lambda pair: pair[0])]
    sv_lengths_sorted = [x for x, _ in sorted(zip(sv_lengths, sv_data), key=lambda pair: pair[0])]

    dim = sv_lengths_sorted[0]
    i = 0

    start_time = time.time()

    while i < len(sv_data):
        i0 = i
        while i < len(sv_data) and sv_lengths_sorted[i] == dim:
            i += 1
        
        sv_use = sv_sorted[i0:i]
        print(experiment_dict["name"], dim, i)

        n_qubits = int(math.log2(dim))

        if n_qubits not in multi_qubit_cliffords:
            multi_qubit_cliffords[n_qubits] = build_clifford_states(n_qubits)
        if n_qubits not in z_bases:
            z_bases[n_qubits] = computational_bases(n_qubits)
        
        rhos = [np.outer(sv, sv.conjugate()) for sv in sv_use]

        y_max = np.array([-1 for _ in sv_use], dtype='float')

        for haar_basis in tqdm(multi_qubit_cliffords[n_qubits]):
            kd_sums = np.array([-1 for _ in sv_use], dtype='float')
            
            for hs in haar_basis:
                Pi_b = np.outer(hs, hs.conjugate())
                kd_sum = np.array([abstrace(np.matmul(Pi_b, rho)) for rho in rhos], dtype='float')
                kd_sums += kd_sum
                
            y_max = y_max*(y_max > kd_sums) + kd_sums*(kd_sums >= y_max)

        for j in range(i0, i):
            NCLs[j] = y_max[j-i0]
        if i < len(sv_data):
            dim = sv_lengths_sorted[i]

    end_time = time.time()

    experiment_book[k]["NCLs"] = NCLs
    experiment_book[k]["coherence"] = sum(NCLs)/len(NCLs)
    coherence_dict[experiment_dict["name"]] = {"name": NCLs, "avg": sum(NCLs)/len(NCLs)}

    print(experiment_dict["name"], NCLs, sum(NCLs)/len(NCLs))
    print(end_time-start_time)

Tagging sentences:   0%|          | 0/15 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/60 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/60 [00:00<?, ?it/s]

generalQC_H_CRx 4 17


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 576/576 [00:00<00:00, 1506.60it/s]


...

Tagging sentences:   0%|          | 0/15 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/60 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/60 [00:00<?, ?it/s]

generalQC_H_CRz 4 17


 22%|█████████████████████▍                                                                            | 126/576 [00:00<00:00, 1255.34it/s]